In [83]:


import pandas as pd
from sklearn.model_selection import GridSearchCV
from joblib import load
import numpy as np
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import random
import json

In [84]:
city = 'hn'
model_name = 'xgb'
version = 5
# model = load(f'/home/long/airflow/dags/models/{city}/{model_name}/v{version}/model.joblib')

In [85]:
df = pd.read_parquet(f'/home/long/long/datn-feast/data/update_data/demo1/realestate_dataset_{city}_v{version}.parquet')
df

,target,event_timestamp,realestate_id,nearest_2_street,houseDirection,nearest_1_ward,nearest_6_district,nearest_3_district,nearest_4_ward,nearest_8_district,...,num_of_school_in_1000m_radius,num_of_school_in_2000m_radius,num_of_marketplace_in_2000m_radius,distance_hcm_PhoHaiBaTrung,distance_nearest_0,nearest_2_lat,facade,num_of_bank_in_2000m_radius,nearest_5_lon,num_of_restaurant_in_2000m_radius
0,122.448980,2024-07-05 03:57:43+00:00,0,3116,5000,360,29,29,91,5000,...,7.0,19.0,10.0,8.977802,6.361190,21.066785,0,6.0,105.773168,201.0
1,168.750000,2024-07-05 03:57:54+00:00,1,978,5000,265,5000,24,265,5000,...,16.0,56.0,8.0,8.553683,6.029161,20.992102,0,24.0,105.818291,51.0
2,250.000000,2024-07-05 03:59:24+00:00,2,1454,5000,445,3,3,445,5000,...,19.0,54.0,6.0,8.184312,4.943376,21.000553,0,37.0,105.862110,70.0
3,157.317073,2024-07-05 04:07:14+00:00,3,957,1,84,5,27,63,5,...,14.0,59.0,16.0,8.515749,5.639214,21.032808,0,48.0,105.796689,161.0
4,38.297872,2024-07-05 04:09:06+00:00,4,10,5000,442,6,6,10,9,...,3.0,10.0,2.0,9.229430,7.401369,21.016497,0,1.0,105.923818,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,162.500000,2024-07-06 03:12:45+00:00,622,2787,5000,155,12,12,155,12,...,17.0,43.0,2.0,9.117892,4.422825,20.971861,0,9.0,105.776924,19.0
623,423.780488,2024-07-06 03:18:20+00:00,623,648,5000,59,19,19,101,19,...,7.0,22.0,3.0,8.793749,6.160887,21.072403,2,5.0,105.812200,41.0
624,35.483871,2024-07-06 03:23:30+00:00,624,1783,5000,538,8,8,622,5000,...,4.0,5.0,1.0,9.730656,7.605199,20.936626,0,0.0,105.731414,0.0
625,129.310345,2024-07-06 03:24:45+00:00,625,767,5000,360,29,29,254,29,...,3.0,17.0,11.0,9.114465,5.948197,21.077481,0,5.0,105.772538,190.0


In [86]:
featureset = json.load(open(f'/home/long/long/datn-feast/data/featureset/{city}_v{version}.json'))

In [87]:
f'/home/long/long/datn-feast/data/featureset/{city}_v{version}.json'

'/home/long/long/datn-feast/data/featureset/hn_v1.json'

In [88]:
order_config = {
    'abr': "num-cat",
    'gbr': "num-cat",
    'etr': 'cat-num',
    'cat': 'cat-num',
    'knr': 'num-cat',
    'linear': 'num-cat',
    'ridge': 'num-cat',
    'mlp': 'num-cat',
    'rf': 'cat-num',
    'xgb': 'cat-num',
    'lgbm': 'cat-num',
    'la': 'num-cat'
}
order = order_config[model_name]

In [89]:
cat_cols = featureset['cat_cols']
num_cols = featureset['num_cols']

In [90]:
if order == "num-cat":
    all_cols = num_cols + cat_cols
else:
    all_cols =  cat_cols + num_cols

In [91]:
X = df[all_cols]
y = df['target']

In [92]:
X

,nearest_1_street,nearest_1_ward,nearest_0_street,nearest_8_ward,nearest_8_street,nearest_6_street,nearest_2_ward,nearest_1_district,nearest_5_street,nearest_0_district,...,num_of_place_of_worship_in_2000m_radius,nearest_2_lat,distance_hcm_VuonHoaPasteur,distance_hcm_Congvienthule,nearest_3_lon,distance_hcm_HoGuom,distance_hcm_HangHanh,num_of_restaurant_in_2000m_radius,distance_nearest_2,distance_hcm_PhoHaiBaTrungHoanKiem
0,2646,360,2146,5000,5000,1722,91,29,3079,29,...,12.0,21.066785,9.193800,8.422253,105.789889,9.000937,8.987714,201.0,6.931080,9.024885
1,416,265,1233,5000,5000,5000,265,24,5000,24,...,12.0,20.992102,8.582750,8.457929,105.817842,8.639754,8.680929,51.0,6.159080,8.582585
2,1067,445,932,5000,5000,1600,46,3,1003,3,...,8.0,21.000553,7.480866,8.860359,105.860349,8.202699,8.266965,70.0,5.617907,8.100231
3,525,84,254,196,270,2389,190,5,1036,5,...,34.0,21.032808,8.809274,6.705458,105.802247,8.582726,8.584067,161.0,6.629978,8.591747
4,1173,442,1055,10,508,2082,10,9,131,9,...,13.0,21.016497,9.110429,9.572067,105.922253,9.187914,9.183001,0.0,7.636529,9.192964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,2866,155,2938,226,3224,979,155,12,2193,12,...,9.0,20.971861,9.155163,8.863078,105.779741,9.169382,9.191010,19.0,4.795688,9.140724
623,584,59,128,59,2886,846,59,19,2369,19,...,10.0,21.072403,9.043514,8.587853,105.814163,8.787623,8.754773,41.0,6.626035,8.837055
624,2565,538,990,5000,5000,947,538,12,2654,37,...,8.0,20.936626,9.762083,9.535979,105.724873,9.758998,9.769379,0.0,7.653060,9.745736
625,2646,360,687,91,522,1982,360,29,1935,29,...,10.0,21.077481,9.306789,8.673453,105.771768,9.131130,9.117374,190.0,6.177637,9.154782


In [93]:
pretrained_file = f'/home/long/airflow/dags/models/{city}/{model_name}/v{version}/model.joblib'

In [94]:
categorical_features_indices = [i for i, c in enumerate(all_cols) if c in cat_cols]

In [95]:
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from joblib import load
import numpy as np
import lightgbm as lgb

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class BKPriceEstimator(RegressorMixin, BaseEstimator):
    def __init__(self, pretrained_model_path, update_model, weight = 0.001, type = None) -> None:
        self.pretrained_model_path = pretrained_model_path
        self.update_model = update_model
        self.weight = weight
        self.type = type

    def fit(self, X, y=None):

        self.X = X
        self.y = y
        print("Start training update model")
        self.update_model.fit(X, y)
        print("End training update model")
        print()
        if self.type == 'lgbm':
            self.pretrained_model = load(self.pretrained_model_path)
            self.pretrained_model.booster_.save_model('./lgbm_cache.txt')
            self.pretrained_model = lgb.Booster(model_file='./lgbm_cache.txt')
        else:
            self.pretrained_model = load(self.pretrained_model_path)
        self.is_fitted_ = True
        return self

    def predict(self, X):
        check_is_fitted(self, ['is_fitted_', 'update_model'])

        base_pred = np.abs(self.pretrained_model.predict(X))
        new_pred = np.abs(self.update_model.predict(X))

        # result = base_pred * (1 - self.weight) + new_pred * self.weight
        result = base_pred

        return result

    def get_params(self, deep = False):
        return {
            'weight':self.weight,
            'pretrained_model_path':self.pretrained_model_path,
            'update_model':self.update_model,
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self





In [96]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import random

from joblib import load

def create_model(pretrained_file = None):
    update_model = XGBRegressor(n_estimators = 1000, random_state=random.randint(3, 1000), verbose=10, learning_rate=0.04, tree_method = 'gpu_hist')
    model = BKPriceEstimator(
        pretrained_model_path = pretrained_file,
        update_model = update_model,
    )
    search_params = {'weight': [-0.001, 0, 0.001]}
    clf = GridSearchCV(model, search_params, scoring=['explained_variance', 'max_error', 'neg_root_mean_squared_error', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_percentage_error'], refit='neg_root_mean_squared_error', cv=5)
    return model

model = create_model(pretrained_file)

In [97]:
model.fit(X, y)

Start training update model


/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:03:35] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:03:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


End training update model



/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:03:43] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


BKPriceEstimator(pretrained_model_path='/home/long/airflow/dags/models/hn/xgb/v1/model.joblib',
                 update_model=XGBRegressor(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.04, max_bin=None,
                                           max_cat

In [98]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [99]:
test_df = pd.read_parquet(f'/home/long/long/datn-feast/data/data_df_{city}_v{version}.parquet')
target_test_df = pd.read_parquet(f'/home/long/long/datn-feast/data/target_df_{city}_v{version}.parquet')

In [100]:
sample_dict = {
    "hcm": 35000,
    "hn": 100000
}

In [101]:
X_test = test_df[all_cols].iloc[sample_dict[city]:]
y_test = target_test_df['target'].iloc[sample_dict[city]:]

In [102]:
X_test

,nearest_1_street,nearest_1_ward,nearest_0_street,nearest_8_ward,nearest_8_street,nearest_6_street,nearest_2_ward,nearest_1_district,nearest_5_street,nearest_0_district,...,num_of_place_of_worship_in_2000m_radius,nearest_2_lat,distance_hcm_VuonHoaPasteur,distance_hcm_Congvienthule,nearest_3_lon,distance_hcm_HoGuom,distance_hcm_HangHanh,num_of_restaurant_in_2000m_radius,distance_nearest_2,distance_hcm_PhoHaiBaTrungHoanKiem
100000,691,86,931,129,641,612,309,24,1338,24,...,14,20.999246,8.399532,8.303108,105.821956,8.440035,8.489295,69,5.823701,8.371868
100001,3874,238,378,100,100,100,238,29,1956,29,...,2,21.050495,9.429021,8.788978,105.740302,9.304599,9.301335,17,6.387527,9.313312
100002,1490,8,1482,86,2027,370,8,7,720,7,...,21,21.010484,8.321973,7.996394,105.820779,8.251390,8.301981,133,5.708239,8.186682
100003,71,45,648,212,554,2805,101,19,180,19,...,15,21.069781,8.993468,8.499091,105.817190,8.722890,8.689022,50,6.226111,8.774286
100004,7,7,398,100,100,100,7,6,100,6,...,19,21.041930,8.203081,8.959883,105.818314,8.128367,8.088522,23,6.002611,8.183439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109493,1237,93,1079,51,1050,56,245,7,541,7,...,39,21.014682,8.211403,7.822959,105.820601,8.007442,8.060743,184,5.742737,7.947803
109494,2187,188,741,376,2387,3569,376,3,2055,38,...,76,21.019843,7.009326,8.479325,105.846757,6.987177,7.214683,706,5.022000,6.526624
109495,1712,52,2550,88,102,2967,52,12,563,12,...,1,20.992327,9.427645,9.025019,105.737516,9.398892,9.411328,1,7.083292,9.383911
109496,2675,194,2194,10,1185,1416,41,9,3999,9,...,5,20.996273,9.327854,9.749260,105.953173,9.455880,9.462720,4,7.473056,9.444235


In [103]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = np.sqrt(mean_squared_error(y_test, y_pred, squared = True))
rsq = r2_score(y_test, y_pred)

print(f'mae {mae:.5f}, rmse {mse:.5f}, rsq {rsq:.5f}')

/home/long/anaconda3/envs/mlops-env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:03:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


mae 33.68756, rmse 55.80694, rsq 0.83583
